In [1]:
# evaluation.py (Test Set 분리 논리 반영)

# ============================
# 0. Colab 환경 설정 및 경로 정의
# ============================
import os
from google.colab import drive

# ⚠️ 1. Google Drive 마운트
drive.mount('/content/drive')

# ⚠️ 2. PROJECT_DIR 경로 설정
# 현재 설정: '/content/drive/MyDrive/Colab Notebooks'
# 이 경로는 textbook_content.csv 파일과 instruction_classifier_model 폴더의 공통 상위 폴더여야 합니다.
PROJECT_DIR = '/content/drive/MyDrive/Colab Notebooks'

# 파일 경로 정의
CSV_FILE_PATH = os.path.join(PROJECT_DIR, "textbook_content.csv")
MODEL_DIR = os.path.join(PROJECT_DIR, "instruction_classifier_model")


# ============================
# 1. 라이브러리 불러오기
# ============================
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# ============================
# 2. 데이터셋 정의 (InstructionDataset)
# ============================
class InstructionDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_len=64):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = str(self.sentences[idx])
        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long),
        }


# ============================
# 3. Test 데이터 불러오기 & 분리
# ============================
print("--- 1단계: 데이터 로드 및 Test set 분리 ---")
try:
    df = pd.read_csv(CSV_FILE_PATH, encoding='cp949')
except UnicodeDecodeError:
    df = pd.read_csv(CSV_FILE_PATH, encoding='utf-8')
except FileNotFoundError as e:
    print(f"❌ 오류: 데이터 파일이 없습니다. 경로를 확인하세요: {CSV_FILE_PATH}")
    print("경로 설정이 올바른지 확인해주세요.")
    exit()

sentences = df["sentence"].tolist()
labels = df["label"].tolist()

# ⚠️ Test set 추출 논리 수정:
# training.py에서 전체 데이터를 80% (Train/Val)와 20% (Test)로 나누는 것과
# 정확히 동일한 방식으로 Test Set을 추출합니다.
train_val_s, test_s, train_val_l, test_l = train_test_split(
    sentences, labels, test_size=0.2, random_state=42, shuffle=True
)

# Test set을 최종 평가에 사용
# (train_val_s와 train_val_l은 버립니다.)
print("Test set 분리를 위해 training.py와 동일한 random_state=42를 사용했습니다.")


# 토크나이저 로드 (⚠️ 로컬 파일 경로 사용 및 온라인 검색 방지)
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, local_files_only=True)

test_dataset = InstructionDataset(test_s, test_l, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=16)

print(f"Test set 크기: {len(test_dataset)}개")


# ============================
# 4. 모델 로드 (재현 가능한 모델 복원)
# ============================
print("\n--- 2단계: 학습된 모델 로드 ---")
try:
    # 모델 로드 (⚠️ 로컬 파일 경로 사용 및 온라인 검색 방지)
    model = BertForSequenceClassification.from_pretrained(MODEL_DIR, local_files_only=True)
    model.to(device)
    print(f"모델 로드 성공: {MODEL_DIR}/")
except OSError:
    print(f"❌ 오류: 모델 파일을 로드할 수 없습니다. 경로를 확인하세요: {MODEL_DIR}")
    print("1. training.py를 먼저 실행하여 모델을 해당 경로에 저장했는지 확인하세요.")
    print("2. MODEL_DIR 경로가 Google Drive의 실제 모델 폴더와 일치하는지 확인하세요.")
    exit()


# ============================
# 5. 최종 평가 함수
# ============================
def final_eval_model(model, loader):
    model.eval()
    preds, trues = [], []

    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask=mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)

            preds.extend(predictions.cpu().numpy())
            trues.extend(labels.cpu().numpy())

    return classification_report(trues, preds, digits=4)


# ============================
# 6. 평가 실행
# ============================
print("\n--- 3단계: Test set 최종 평가 수행 ---")
print("평가 지표 (Precision, Recall, F1-score, Support) :")
evaluation_report = final_eval_model(model, test_loader)

print("="*60)
print(evaluation_report)
print("="*60)
print("\n평가 완료.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
--- 1단계: 데이터 로드 및 Test set 분리 ---
Test set 분리를 위해 training.py와 동일한 random_state=42를 사용했습니다.
Test set 크기: 62개

--- 2단계: 학습된 모델 로드 ---
모델 로드 성공: /content/drive/MyDrive/Colab Notebooks/instruction_classifier_model/

--- 3단계: Test set 최종 평가 수행 ---
평가 지표 (Precision, Recall, F1-score, Support) :
              precision    recall  f1-score   support

           0     0.8000    1.0000    0.8889        36
           1     1.0000    0.6538    0.7907        26

    accuracy                         0.8548        62
   macro avg     0.9000    0.8269    0.8398        62
weighted avg     0.8839    0.8548    0.8477        62


평가 완료.
